In [25]:
import numpy as np
import random
import re

from numpy import asarray
from typing import NamedTuple
from PIL import Image

img = []
ternary_img = []
img_width = 994

token_pure_specification = [
    ('ALL_ONES', r'1+$'),
    ('ALL_ZEROS', r'0+$'),
    ('ALT_10', r'(10)+$'),
]

ternary_token_specification = [
    ('ALT_10', r'^(10)+'),
    ('ALL_ONES', r'^1+'),
    ('ALT_21', r'^(21)+'),
    ('ALT_20', r'^(20)+'),
#    ('ALT_10', r'^(10)+'),
    ('ALL_ZEROS', r'^0+'),
    ('ALL_TWOS', r'^2+'),
]

class Token(NamedTuple):
    type: str
    value: str
    carry: bool
    next_value: str

def tokenize(code):
    tok_pure_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_pure_specification)
    match = re.search(tok_pure_regex, code)
    while match:
        kind = match.lastgroup
        value = match.group()
        yield Token(kind, value, False, '')
        code = code[:match.start()]
        match = re.search(tok_pure_regex, code)
        
def ternary_tokenize(code):
    tok_pure_regex = '|'.join('(?P<%s>%s)' % pair for pair in ternary_token_specification)
    match = re.search(tok_pure_regex, code)
    while match:
        kind = match.lastgroup
        value = match.group()
        yield Token(kind, value, False, '')
        code = code[match.end():]
        match = re.search(tok_pure_regex, code)

def colorPrint(t, next=False, show = True, create_png = False):
    t_reverse = t[::-1]
    pixels = []
    for index, token in enumerate(t_reverse): # index is not what you'd expect!
        if token.type == 'ALL_ONES':
            color = 32 # green
            if (len(token.value) == 1) and (index != 0) and (index != len(t) - 1) \
                and (t_reverse[index - 1].type == 'ALL_ZEROS') and \
                (t_reverse[index + 1].type == 'ALL_ZEROS'):
                bkColor = 41 # 46 # Cyan   # New
            else:
                bkColor = 41 # red
        elif token.type == 'ALL_ZEROS':
            color = 33 # Yellow
            bkColor = 44 # blue
        elif token.type == 'ALT_01':
            color = 36 # cyan
            bkColor = 45 # purple
        else:
            color = 30 # black
            bkColor = 42 # Green 
        if next:
            value = token.next_value
        else:
            value = token.value
            
        if show:
            print(f"\033[2;30;{bkColor}m{value}", end = '')
        if create_png:
            if bkColor == 41:
                pixel = [255, 0, 0] # red
            elif bkColor == 42:
                pixel = [255, 255, 0] # [50, 180, 25] # green
            elif bkColor == 44:
                pixel = [0, 0, 255] # blue
            elif bkColor == 46:
                pixel == [0, 255, 255] # cyan
            elif bkColor == 45:
                pixel = [255, 0, 255] # magenta
            else:
                pixel = [255, 255, 0] # yellow
            cell = [pixel] * len(value)
            cell = [item for sublist in cell for item in sublist]
            pixels.append(cell)
    if create_png:    
        pixels = [item for sublist in pixels for item in sublist]
    return pixels
            
def ternary_colorPrint(t, next=False, show = True, create_png = False):
    t_reverse = t # from left to right, unlike the binary rep
    pixels = []
    for index, token in enumerate(t_reverse): # index is not what you'd expect!
        if token.type == 'ALL_ONES':
            color = 32 # green
            bkColor = 42 # Green 
        elif token.type == 'ALL_ZEROS':
            color = 33 # Yellow
            bkColor = 44 # blue
        elif token.type == 'ALL_TWOS':
            color = 36 # cyan
            bkColor = 41 # red
        elif token.type == 'ALT_10':
            color = 30 # black
            bkColor = 43 # yellow
        elif token.type == 'ALT_20':
            color = 30 # black
            bkColor = 45 # purple
        else:
            color = 30 # black
            bkColor = 46 # cyan
        if next:
            value = token.next_value
        else:
            value = token.value
            
        if show:
            print(f"\033[2;30;{bkColor}m{value}", end = '')
        if create_png:
            if bkColor == 41:
                pixel = [255, 0, 0] # red
            elif bkColor == 42:
                pixel = [0, 255, 0] # [50, 180, 25] # green
            elif bkColor == 44:
                pixel = [0, 0, 255] # blue
            elif bkColor == 46:
                pixel = [0, 255, 255] # cyan
            elif bkColor == 45:
                pixel = [255, 0, 255] # magenta
            else:
                pixel = [255, 255, 0] # yellow
            cell = [pixel] * len(value)
            cell = [item for sublist in cell for item in sublist]
            pixels.append(cell)
    if create_png:    
        pixels = [item for sublist in pixels for item in sublist]
    return pixels

def breakDownAndPrint(binRep, 
                      colorTernaryLsb = False, 
                      show = True, 
                      m = 0, 
                      K = 0, 
                      first_time = True, 
                      create_png = False, 
                      offset = 0, 
                      ternary_offset = 0,
                      use_ternary = True,
                      simplified_collatz = True):
    
    global img_row
    global img
    
    s = []
    t = []
    for token in tokenize(binRep):
        s.append(token.value)
        t.append(token)
    pixels = colorPrint(t, show = show, create_png = create_png)

    binRepLen = len(binRep)
    number = int(binRep, 2)
    numberStr = str(number)
    numberStrLen = len(numberStr)
    numberSpace = 56 - binRepLen - numberStrLen
    numberStrSpaced = ' ' * numberSpace + numberStr
    ternary = np.base_repr(number,base=3)
    
    ternarySpace = 28 - len(ternary)
    
    if show:
        print(f"\033[2;30;{47}m{numberStrSpaced:} ", end = '')
    s3 = []
    t3 = []
    for token in ternary_tokenize(ternary):
        s3.append(token.value)
        t3.append(token)

    ternaryStr = ' ' * ternarySpace    
    # If binRep ends with an even number of blues/zeros,
    # then highlight the last 2 of ternary with BLUE
    # and highlight the last 2 of the next ternary also 
    # with GREEN. 
    # If binRep ends with an odd number of blues/zeros,
    # then highlight the last 2 of the ternary with YELLOW,
    # and highlight the last 1 of the next ternary as CYAN,
    # and the last 2 of the third ternary as GREEN again,
    # indicating that we had a division by an odd power of 
    # 2, that took us to the n%3=1 set, and then from there we 
    # immediately went back to n%3=2. 
    #
    # We can even verify that this sequences always happen, 
    # by predicting it, and then verifying them.
    # For this, breakDownAndPrint can receive a state, and 
    # return a state, and verify the state transition.
    #
    # states are:
    # s2_odd: number is odd and is in the n%3=2 set. It has a 1
    #    at the lsb side of its binary representation), and it is 
    #    in the n%3=2 set (the last digit of its ternary is 2).
    # s2_even_odd: number is even, and is divisible by an odd power
    #    of 2 (it has an odd number of 0's on the lsb side of its
    #    binary representation), and it is in the n%3=2 set,
    #    (the last digit of its ternary is 2). 
    #    When it is wooshed down (all of its 2 factors are removed), 
    #    it becomes a n%3=1 number; also an odd number, s1_odd (n%2=1). 
    # s1_odd: number is odd (has a 1 at the lsb of its binary representation), 
    #    and it is in the n%3=1 set (the lsb digit of its ternary
    #    representation is 1).
    # s2_even_even: number is even, and is divisible by an even power
    #    of 2 (it has an even number of 0's on the lsb side of its
    #    binary representation), and it is in the n%3=2 set (the last 
    #    digit of its ternary is 2).
    #    When it is wooshed down (all of its 2 factors are removed), 
    #    it stays a n%3=2 number; also an odd number, s2_odd (n%2=1).
    # There is no s1_even.
    # s3: number is divisble by 3, and is in the n%3=0 set, and the lsb 
    #    of its ternary representation is 0. This number may also be 
    #    divisible by a power of 2. Wooshing it down will move it either
    #    to a s1_odd state, or to a s2_odd state, accordingly. 
    # 
    # So the transitions after the application of the collatz rules are:
    # s3 -> s1_odd | s2_odd 
    # s2_odd -> s2_odd | s2_even_odd | s2_even_even 
    # s2_even_odd -> s1_odd
    # s2_even_even -> s2_odd
    # s1_odd -> s2_odd | s2_even_odd
    # 
    # priting:
    # In s3: use GREEN for lsb of ternary (0). Return a flag to color the lsb of next ternary.
    # In s1_odd: use CYAN for lsb of ternary. Return a flag to color the lsb of next ternary.
    # In s2_even_odd: use YELLOW for lsb of ternary.
    # If in s2_odd, only if received a flag to color, then color the lsb of ternary with PURPLE.
    # If in s2_even_even: use BLUE for LSB of ternary. Return a flag to color the lsb of next ternary.    
    
    # Let's find out the lsb of binary, the lsb of ternary, the divisibility by a power of 2.
    # These will help us determin the current state, to decide how to paint the last ternary
    # (also given the input flag), and what flag to return.
    
    # Have this method return the maximal number of 1's, m, in the lsb of binRep, together with K,
    # where K = (n - (2^m - 1))/(2^m), (binary)n = [(binary)K][1]*m. Note that K is even and its lsb = 0.
    # When the caller receives the first non-zero m, let's call it m*, it stores m* and the corresponding K*
    # until a later m that it receives is 0. At that point, the caller sets a flag to remind itself
    # that it should store the next m* and K*. The caller keeps sending its stored
    # m* and K* back to this function, and asks this function to check if the next m that this function
    # computes and is about to return is 0, and if so, compare the received K* with the following number:
    # Q = (p - (3^m* - 1))/(3^m*), and assert that Q == K*, (ternary)p = [(ternary)Q][2]*m, and also print
    # Q.
    # We also ask this function that whenever it receives a m = 0 from the caller (which means that in the
    # previous call, this function produces a 0 at the lsb of n), that it prints the K that it computes.
    
    ternaryLsb = ternary[-1]
    binaryLsb = binRep[-1]
    s3 = ternaryLsb == '0'

    if s3:
        if show:
            print(f"\033[2;30;{42}m{ternaryStr}", end = '') # '\n') # GREEN
        colorTernaryLsb = True
    else:
        s2 = ternaryLsb == '2'
        odd = binaryLsb == '1'
        if odd:
            if s2: 
                s2_odd = True
                if colorTernaryLsb:
                    if show:
                        print(f"\033[2;30;{45}m{ternaryStr}", end = '') # PURPLE
                else:
                    if show:
                        print(f"\033[2;30;{47}m{ternaryStr}", end = '') # WHITE
                colorTernaryLsb = False
            else: # s1
                s1_odd = True
                if show:
                    print(f"\033[2;30;{46}m{ternaryStr}", end = '') # CYAN
                colorTernaryLsb = True
        else: # s2_even
            tok_regex = '(?P<%s>%s)' % ('ALL_ZEROS', r'0+$')
            match = re.search(tok_regex, binRep)
            assert(match)
            s2_even_even = len(match.group()) % 2 == 0
            if s2_even_even:
                if show:
                    print(f"\033[2;30;{44}m{ternaryStr}", end = '') # BLUE
                colorTernaryLsb = True
            else:
                if show:
                    print(f"\033[2;30;{43}m{ternaryStr}", end = '') # YELLOW
                colorTernaryLsb = False
                
    ternary_pixels = ternary_colorPrint(t3, show = show, create_png = create_png)
    if create_png:
        post_pad = [128, 128, 128] * offset
        if not use_ternary:
            pre_pad = [128, 128, 128] * (img_width - offset - 2 * len(pixels)//3)
            pixels = pixels + pre_pad + pixels + post_pad
        else:
            pre_pad = [128, 128, 128] * (img_width - offset - len(pixels)//3)
            pixels = pre_pad + pixels + post_pad
            ternary_pre_pad = [128, 128, 128] * ternary_offset
            ternary_post_pad = [128, 128, 128] * (img_width - ternary_offset - len(ternary_pixels)//3)
            ternary_pixels = ternary_pre_pad + ternary_pixels + ternary_post_pad
            pixels = list([p if p != 128 else ternary_pixels[index] for index, p in enumerate(pixels)])
        img.append(pixels)

    if create_png:
        ternary_img.append(ternary_pixels)

    ternaryLen = len(ternary)
    binaryLen = len(binRep)
    lenDiff = len(ternary) - len(numberStr)
    if show:
        print (f"  {binaryLen:2}, {ternaryLen:2}", end = '')
#        print (f"  {binaryLen:2}, {ternaryLen:2}, {lenDiff:2}, {binaryLen/ternaryLen:.4f}", end = '')
        
    if (t[0].type == 'ALL_ONES'):
        new_m = len(t[0].value)
        two_to_m = 1 << new_m
        new_K = (number - two_to_m + 1) // two_to_m
    elif (t[0].type == 'ALT_01'):
        new_m = 1
        two_to_m = 1 << new_m
        new_K = (number - two_to_m + 1) // two_to_m
    else:
        new_m = 0
        new_K = 0
        three_to_m = int(('1' + '0' * m), 3)
        if not first_time:
            Q = (number - three_to_m + 1) // three_to_m
            if simplified_collatz:
                if (Q != K):
                    print(f"\nQ = {Q}, K = {K}, m = {m}, new_m = {new_m}, new_K = {new_K}")
                assert(Q == K)
            if show:
                print(f" {Q}", end = '')
    if m == 0:
        if show:
            print(f" {new_K}", end = '')
    if show:
         print("")    
    return s, t, colorTernaryLsb, new_m, new_K

def generate_image(number, img, reflect = True):
    img = np.array(img, np.uint8)
    print(img.shape)
    img = np.reshape(img, (img.shape[0], -1, 3))

    height = img.shape[0]
    width = img.shape[1]

    img2 = np.array(img)
    if reflect:
        for i in range(height):
            for j in range(width):
                ip = height - j - 1
                jp = width - i - 1
                if img[i][j][0] == 128 and img[i][j][1] == 128 and img[i][j][2] == 128:
                    break
                color = img[i][j]
                if color[1] != 0:
                    if not use_ternary:
                        img2[i][j] = [0, 255, 0]
                        img[i][j] = [0, 255, 0]
                    else:
                        img2[i][j] = img[i][j]                    
                if ip >= 0 and ip < height and jp >= 0 and jp < width and \
                    (img[i][j][0] != 128 or img[i][j][1] != 128 or img[i][j][2] != 128) and \
                    img[ip][jp][0] == 128 and img[ip][jp][1] == 128 and img[ip][jp][2] == 128:
                    if color[1] != 0:
                        img2[ip][jp] = [0, 255, 255] # cyan
                    else:
                        img2[ip][jp] = color
            
    def colored_block(c):
        height = 20
        width = 16
        return np.full((height, width, 4), c, dtype=np.uint8)

    red_block = colored_block((255, 0, 0, 255))
    green_block = colored_block((0, 255, 0, 255))
    blue_block = colored_block((0, 0, 255, 255))
    yellow_block = colored_block((255, 255, 0, 255))
    magenta_block = colored_block((255, 0, 255, 255))
    cyan_block = colored_block((0, 255, 255, 255))

    collatz_image = np.ones((height,width,20,16,4), np.uint8) * 100

    for i in range(height):
        for j in range(width):
            if np.all(img2[i, j] == [255, 0, 0]):
                collatz_image[i,j] = red_block
            elif np.all(img2[i, j] == [0, 255, 0]):
                collatz_image[i,j] = yellow_block
            elif np.all(img2[i, j] == [0, 0, 255]):
                collatz_image[i,j] = blue_block
            elif np.all(img2[i, j] == [255, 0, 255]):
                collatz_image[i,j] = magenta_block
            elif np.all(img2[i, j] == [255, 255, 0]):
                collatz_image[i,j] = green_block
            elif np.all(img2[i, j] == [0, 255, 255]):
                collatz_image[i,j] = cyan_block 
            else:
                collatz_image[i, j, :, :, :] = 255 

    collatz_image = np.concatenate(collatz_image, axis=1)
    collatz_image = np.concatenate(collatz_image, axis=1)

    collatz_image = Image.fromarray(collatz_image)
    collatz_image.save('CollatzImage_' + str(number) + '.png')
    print(f"Saved CollatzImage_{str(number)}.png")

def computeNextCollatz(t, carry = True): 
    t_next = []
    s_next = []
    for index, token in enumerate(t):
        previous_carry = carry
        l = list(token.value)
        if token.type == 'ALL_ONES':
            if previous_carry:
                l[0] = '0'
                carry = True
            else:
                if (len(l) > 1):
                    l[0] = '0'
                    l[-1] = '0'
                    carry = True
                else:
                    # no change to l
                    carry = False                
        elif token.type == 'ALL_ZEROS':
            carry = False
            if previous_carry:
                l[0] = '1'
                l[-1] = '1'
            else:
                l[0] = '1'
        elif token.type == 'ALT_10':
            if (index == len(t) - 1) or (t[index+1].value[-1] == '0'):
                if previous_carry:
                    l = ['0' for c in l]
                    carry = True
                else:
                    l = ['1' for c in l]
                    carry = False
            else:
                if previous_carry:
                    l = ['0' for c in l]
                    l[0] = '1'
                    carry = True
                else:
                    l = ['1' for c in l]
                    l[0] = '0'
                    carry = True            
        else: # 'ALT_01'
            if (index == len(t) - 1) or (t[index+1].value[-1] == '0'):
                if previous_carry:
                    l = ['0' for c in l]
                    l[0] = '1'
                    carry = False
                else:
                    l = ['1' for c in l]
                    l[0] = '0'
                    carry = False
            else:
                if previous_carry:
                    l = ['0' for c in l]
                    carry = True
                else:
                    l = ['1' for c in l]
                    carry = False
                    
        t_next.append(Token(token.type, token.value, previous_carry, ''.join(l)))
        s_next.append(''.join(l))
    if carry:
        t_next.append(Token('ALL_ZEROS', '0', False, '1'))
        s_next.append('1')
        
    return s_next, t_next, carry        

def simulateCollatz(number, create_png = False, 
                    show = True, 
                    show_div_by_two = True,
                    use_ternary = True, 
                    simplified_collatz = True,
                    offset = 1, 
                    ternary_offset = 1):
    assert(number >= 1)
    colorTernaryLsb = False
    reds = 0
    blue_greens = 0
    m = 1
    K = 0
    reset_m = True
    first_time = True
    steps = 0
    next_ternary_offset = ternary_offset

    while True:

        binRep = format(number, "b")
        bin_rep_len = len(binRep)
        ternary_rep = np.base_repr(number, base=3)
        ternary_rep_len = len(ternary_rep)
        #print(f"\033[2;30;47mnumber is: {number} \n{binRep}")
        s, t, colorTernaryLsb, new_m, new_K = breakDownAndPrint(binRep, 
                                                                colorTernaryLsb = colorTernaryLsb, 
                                                                m = m, 
                                                                K = K, 
                                                                first_time = first_time, 
                                                                show = show,
                                                                create_png = create_png,
                                                                offset = offset,
                                                                ternary_offset = next_ternary_offset,
                                                                use_ternary = use_ternary,
                                                                simplified_collatz = simplified_collatz)
        first_time = False
        if reset_m and new_m > 0:
            m = new_m
            K = new_K
            reset_m = False
        elif new_m == 0:
            reset_m = True
        modified = False
        
        if simplified_collatz:
            while number % 2 == 0:
                modified = True
                number = number // 2
                blue_greens += 1
        elif number % 2 == 0:
            modified = True
            number = number // 2
            blue_greens += 1
        offset += (bin_rep_len - len(format(number, "b")))
        next_ternary_rep_len = len(np.base_repr(number,base=3))
        next_ternary_offset += (ternary_rep_len - next_ternary_rep_len)
        
        if number % 2 != 0:
            next_number = int((3 * number + 1) // 2)
            nextBinRep = format(next_number, "b")
            reds += 1
        else:
            next_number = number
            nextBinRep = binRep            
        #print(f"next   is: {next_number}, \n{nextBinRep}")

        # Note that s/t/s_next/t_next are all inverted
        if modified:
            binRep = format(number, "b")
            s, t, colorTernaryLsb, new_m, new_K = breakDownAndPrint(binRep, 
                                                                    colorTernaryLsb = colorTernaryLsb, 
                                                                    show = show_div_by_two or number == 1,
                                                                    m = m, 
                                                                    K = K, 
                                                                    first_time = first_time,
                                                                    create_png = number == 1, # False
                                                                    offset = offset,
                                                                    ternary_offset = next_ternary_offset,
                                                                    use_ternary = use_ternary,
                                                                    simplified_collatz = simplified_collatz)
            if reset_m and new_m > 0:
                m = new_m
                K = new_K
                reset_m = False
            elif new_m == 0:
                reset_m = True
                
            if number == 1:
                print(f"reds = {reds}, blue_greens = {blue_greens}")
                break
        ternary_offset -= (len(np.base_repr(next_number,base=3)) - next_ternary_rep_len + 1)
        s_next, t_next, carry = computeNextCollatz(t, carry = True) # carry = carry) # 10/2/2021
                
        next_number_predict = int(''.join(s_next[::-1]), 2)   

        #colorPrint(t_next, next=True)
        nextBinRep = format(next_number_predict, "b")
        #s_update, t_update = breakDownAndPrint(nextBinRep)
        
        if simplified_collatz:
            assert(next_number_predict == next_number)
        else:
            next_number_predict = next_number
        number = next_number_predict

        
number = int("11010110101101011010100011111",2) # 792127658379 # 79126879 # 
number = int("110101101011010110101101011111",2) # 792127658379 # 79126879 # 
number = int("1111001000011111",2) # 792127658379 # 79126879 # 
number = int("1111100001101011000011111",2) # 792127658379 # 79126879 # 
number = int("100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111", 2)
number = int("10000001001010010111011111100100010001111110001111111111110000000111111111111000001110001001001010011111001010001000000100100000111111111111111111100000001000010000000000000000000000000000000000000011111111100101001010101001010101010100101010100011111111111111111111100000110011111111111111111111111111111111111111110000010100111111111111100000001111111111111111111111111111111111111111111111", 2) # 9756757367240986767370410940184976786720100109385378392818182121128007097968680129483827646308928481800734720948628914673983901035# 792127658379 # 27 + 54 * 3 # 2287 * 8192 # 1407 # 3007 * 8 # int("111110000001",2) # 792127658379 # 79126879 # 
#number = int("0011111111111111111111111111111111111111110000010100111111111111100000001111111111111111111111111111111111111111111111", 2) # 9756757367240986767370410940184976786720100109385378392818182121128007097968680129483827646308928481800734720948628914673983901035# 792127658379 # 27 + 54 * 3 # 2287 * 8192 # 1407 # 3007 * 8 # int("111110000001",2) # 792127658379 # 79126879 # 
number = 792127658379 # + 2**40
#number = int("111110001111100011111000111110001111100011111", 2) # 27 # int("101010101111111101010101010101111110101010101111111", 2)
#number = int("110011001100110011001100110011001100110011001100110011001100", 2)
#number = int("110000000000000000000000111100", 2)
number = 27 # + 2**74
number = int("11111111111110111111111111101111111111111", 2)
number = int("1111111111111011111111111110111111111111", 2)
#number = int("110111010111011110111101101100011011100", 2)
#number = int("110111010111011110111101101100011011101", 2) #works1
#number = int("110111010111011110111101101100011011100", 2) #works2
#number = int("1101110101110111101111011011000110111011", 2) # didn't work
#number = int("1101110101110111101111011011000110111010", 2) # didn't work
#number = int("11011101011101111011110110110001101110111", 2) # work
#number = int("11011101011101111011110110110001101110110", 2) # work
#number = int("1101110101110111101111011011000110111011111", 2) # works
#number = int("1101110101110111101111011011000110111011110", 2) # works
#number = int("110111010111011110111101101100011011101111", 2) # didn't work
#number = int( "10111010111011110111101101100011011101111", 2) # didn't work  # nice
#number = 83076749736483233710327388035874815 + 2**400
#number = int("10111", 2) #27 #3**11 # 27 # 243 # int("10101010110", 2) #27 # * 2 + 1 # 6 * 23 + 1 + 12 * 101
#number = int("111111111111111111111000000000000000000000000000000111000000000000000000000000001111111111111111111111111111111111000111111111111111111111111111111111111000000000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111111111111111111111111100000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111", 2)
#number = 667 #27#478
#number = 2**250 - 1 #792127658379 # + 2**40
#number = int("111100000000000101111111", 2)
number = int("110111010111011110111101101100011011101", 2)   # Pretty X


print(f"number: {number}")
print(f"number%3: {number%3}")

# img_width: 994 for the very large number
# offset is the number of empty pixels on the right side of the binary
# offset: 0 for the very large number
# ternary_offset is the number of empty pixels on the left of the ternary
# ternary_offset: 0 for the very large number

# VIP: Depending on the width of the binary and ternary trajectories,
# they may not fit in the img_width that you specify, and they may also
# overlap. So, if the script fails, increase the img_width and rerun.
    
img_width = 200 
offset = 0 
ternary_offset = 0

show = True                # Produce the progression in text form
create_png = True          # Produce the progression in a PNG file, Collatz.sim
show_div_by_two = False    # Shows the /2 step following the 3n+1 step
use_ternary = True         # Renders ternary progression in the png/crypto img, instead of the binary on left/bottom
simplified_collatz = True  # collapses divide by powers of 2 before 3n+1
simulateCollatz(number, 
                create_png = create_png, 
                show = show,
                show_div_by_two = show_div_by_two,
                use_ternary = use_ternary, 
                simplified_collatz = simplified_collatz,
                offset = offset,
                ternary_offset = ternary_offset)

if create_png:
    generate_image(number, img, reflect = False)


number: 475598346461
number%3: 2
110111010111011110111101101100011011101     475598346461    1200110121021012021110122  39, 25
1010011000011001110011100100010101001100    713397519692    2112012101220021220200222  40, 25 237799173230
11111001001001101011010101100111111101      267524069885     221120112012201211202122  38, 24
101110101101110100001000000011011111100     401286104828    1102100210022121021010222  39, 25 44587344980
10001100001001011100011000001010011111      150482289311     112101102100222100102122  38, 24
11010010001110001010100100001111101111      225723433967     210120122001110112010222  38, 24
100111011010101001111110110010111100111     338585150951    1012100221120200210012222  39, 25
111011000111111110111110001100011011011     507877726427    1210112220212112200022222  39, 25
1011000101011111110011101010010101001001    761816589641    2200211101022022111222222  40, 25
10000101000001111101101011111011111101110  1142724884462   11001020120110110202222222  41, 26 15

This table shows some of the available formats;

TEXT COLOR	CODE	    TEXT STYLE	CODE	BACKGROUND COLOR	CODE
Black	    30	        No effect	0	    Black	            40
Red	        31	        Bold	    1	    Red	                41
Green	    32	        Underline	2	    Green	            42
Yellow	    33	        Negative1	3	    Yellow	            43
Blue	    34	        Negative2	5	    Blue	            44
Purple	    35			                    Purple	            45
Cyan	    36			                    Cyan	            46
White	    37			                    White	            47


In [ ]:
n = 7921273752762702679823918662658379
steps = 0
while n != 1:
    if n % 4 == 0:
        n = n // 4
    else:
        n = 5 * n + 3  # works! 3*n-3, 3*n+3, 3*n-6, 3*n+6
    print(n, " ")
    steps += 1
print(f"took {steps} steps")